In [9]:
import os

from pathlib import Path

In [7]:
os.chdir(Path.cwd().parent)

In [5]:
Path.cwd()

PosixPath('/Users/jochen/projects/fastdeploy/notebooks')

In [8]:
from app.models import Step

# Start Deployment

What should start deployment do?

* Create a deployment model with created = now() and finished = None timestamps
* Look up the steps from last deployment and recreate them as "pending" with new deployment id
* If there are no last steps, create one new dummy step
* Start deployment task
* Mark the first step as "running"

# Finish Step

What should finish step do?

* If the currently running step != the finished step, mark the currently running step as pending again and create a new finished step
* If the currently running step == the fninshed step, mark the currently running step as finished and set the next pending step to "running"

# Finish Deployment

What should happen on finish deployment?

* If the deployment has failed, do nothing
* If the deployment was successful, remove all pending steps